# **MIP4-Run1**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [4]:
problem = """You are task with maximizing the water flow in a network of pipes over 10 time steps. The water enters the system at point A and exits the system at point G. Hence, you need to maximize the amount of water that flows to point G. The water enters point A according to the function f(t) = max(-4/30t^3+t^2-0.234t+3, 0), where t is the time step. There are serveral points that are connected through pipes. Each pipe has a fixed capacity. Each point has to possibility to release water out of the system in case the outgoing pipes are full. There are 12 available pipes in total but only 10 of them can be activated. Deactivated pipes can not be used to transport water. The activation status of a pipe is set before any water arrives at point A and does not change. Finally, water can not be stored at the connection points which means that any water coming in at time t needs to be flowing out at time t + 1.

These are the pipe capacities:
AB: 3
AC: 6
AF: 1
BC: 3
BD: 10
BE: 4
CE: 4
CF: 4
DE: 5
EG: 5
FB: 12
FG: 7 """

In [5]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [6]:
print(response1.choices[0].message.content)

Parameters:
- t = time step
- f(t) = inflow of water at point A 
- C = [3, 6, 1, 3, 10, 4, 4, 4, 5, 5, 12, 7] = Capacities of the 12 pipes in the order AB, AC, AF, BC, BD, BE, CE, CF, DE, EG, FB, FG 

Decision Variables:
Let's denote the 12 pipes with a binary variable X[i], for i=1,...,12. If X[i] = 1, the pipe is activated, if X[i] = 0, the pipe is deactivated.
Let's denote by W[i][t], for i=1,...,12 and t=1,...,10, the amount of water which is flowing through pipe i at time t.

Objective Function:
The aim is to maximize the amount of water flowing to point G. We will also assume we want to do this as soon as possible. The objective function becomes:

    max sum(t * W[10][t] + W[12][t]), for t in range(1, 11)

Subject to the following constraints:
1. Pipe activation constraint: sum(X[i]) for i in range(1, 13) <= 10
2. Inflow pipe capacity constraints: f(t) * X[1] <= W[1][t] <= C[1] * X[1], for t in range(1, 11)
3. Other pipe capacity constraints:
   - for i in [2,3] and t in range(1

### **Generate Pyomo Code**

In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [8]:
Markdown(response2.choices[0].message.content)

Here is the Python code using Pyomo to formulate the problem:

```python
from pyomo.environ import *

# Pipes capacities
C = [3, 6, 1, 3, 10, 4, 4, 4, 5, 5, 12, 7] 

# Sample function for inflow of water at point A
f = lambda t: t*2

# Create model
model = ConcreteModel()

# Sets
model.I = RangeSet(1,12)
model.T = RangeSet(1,10)

# Variables
model.X = Var(model.I, within=Binary)
model.W = Var(model.I, model.T, within=NonNegativeReals)

# Objective
def obj_rule(model):
    return sum(t * model.W[10,t] + model.W[12,t] for t in model.T)
model.obj = Objective(rule=obj_rule, sense=maximize)

# Constraints
def pipe_activation_rule(model):
    return sum(model.X[i] for i in model.I) <= 10
model.pipe_activation = Constraint(rule=pipe_activation_rule)

def inflow_pipe_rule(model, t):
    return model.f(t)*model.X[1] <= model.W[1,t] <= C[0]*model.X[1]
model.inflow_pipe = Constraint(model.T, rule=inflow_pipe_rule)

def other_pipe_rule1(model, i, t):
   if i in (2,3):
       return 0 <= model.W[i,t] <= C[i-1]*model.X[i]
model.other_pipe_rule1 = Constraint(model.I, model.T, rule=other_pipe_rule1)

def other_pipe_rule2(model, i, t):
   if i in (4,5,6,11):
       return sum(model.W[i-1,t-1], model.W[10,t-1]) <= model.W[i,t] <= C[i-1]*model.X[i]
model.other_pipe_rule2 = Constraint(model.I, model.T, rule=other_pipe_rule2)

def other_pipe_rule3(model, i, t):
   if i in (7,8):
       return sum(model.W[1,t-1], model.W[3,t-1]) <= model.W[i,t] <= C[i-1]*model.X[i]
model.other_pipe_rule3 = Constraint(model.I, model.T, rule=other_pipe_rule3)

def other_pipe_rule4(model, i, t):
   if i in (9,10):
       return model.W[4,t-1] <= model.W[i,t] <= C[i-1]*model.X[i]
model.other_pipe_rule4 = Constraint(model.I, model.T, rule=other_pipe_rule4)

def other_pipe_rule5(model, t):
   return sum(model.W[6,t-1], model.W[8,t-1]) <= model.W[9,t] <= C[9]*model.X[10]
model.other_pipe_rule5 = Constraint(model.T, rule=other_pipe_rule5)

def other_pipe_rule6(model, t):
   return sum(model.W[2,t-1], model.W[7,t-1]) <= model.W[11,t] <= C[11]*model.X[12]
model.other_pipe_rule6 = Constraint(model.T, rule=other_pipe_rule6)

# Conservation of flow at junctions and time conservation constraints needs to be implemented based on the specific problem settings
```

Note!
It's assumed that the initial state at time t=0 is zero, and indices are 1-based.

Pyomo can connect to a variety of solvers to compute an answer. To solve the problem you might need to setup a solver in the python environment.

#
### **Run the code Generated by GPT4**

In [33]:
from pyomo.environ import *

# Pipes capacities
C = [3, 6, 1, 3, 10, 4, 4, 4, 5, 5, 12, 7]

# Sample function for inflow of water at point A
f = lambda t: t*2

# Create model
model = ConcreteModel()

# Sets
model.I = RangeSet(1,12)
model.T = RangeSet(1,10)

# Variables
model.X = Var(model.I, within=Binary)
model.W = Var(model.I, model.T, within=NonNegativeReals)

# Objective
def obj_rule(model):
    return sum(t * model.W[10,t] + model.W[12,t] for t in model.T)
model.obj = Objective(rule=obj_rule, sense=maximize)

# Constraints
def pipe_activation_rule(model):
    return sum(model.X[i] for i in model.I) <= 10
model.pipe_activation = Constraint(rule=pipe_activation_rule)

def inflow_pipe_rule(model, t):
    return model.f(t)*model.X[1] <= model.W[1,t] <= C[0]*model.X[1]
model.inflow_pipe = Constraint(model.T, rule=inflow_pipe_rule)

def other_pipe_rule1(model, i, t):
   if i in (2,3):
       return 0 <= model.W[i,t] <= C[i-1]*model.X[i]
model.other_pipe_rule1 = Constraint(model.I, model.T, rule=other_pipe_rule1)

def other_pipe_rule2(model, i, t):
   if i in (4,5,6,11):
       return sum(model.W[i-1,t-1], model.W[10,t-1]) <= model.W[i,t] <= C[i-1]*model.X[i]
model.other_pipe_rule2 = Constraint(model.I, model.T, rule=other_pipe_rule2)

def other_pipe_rule3(model, i, t):
   if i in (7,8):
       return sum(model.W[1,t-1], model.W[3,t-1]) <= model.W[i,t] <= C[i-1]*model.X[i]
model.other_pipe_rule3 = Constraint(model.I, model.T, rule=other_pipe_rule3)

def other_pipe_rule4(model, i, t):
   if i in (9,10):
       return model.W[4,t-1] <= model.W[i,t] <= C[i-1]*model.X[i]
model.other_pipe_rule4 = Constraint(model.I, model.T, rule=other_pipe_rule4)

def other_pipe_rule5(model, t):
   return sum(model.W[6,t-1], model.W[8,t-1]) <= model.W[9,t] <= C[9]*model.X[10]
model.other_pipe_rule5 = Constraint(model.T, rule=other_pipe_rule5)

def other_pipe_rule6(model, t):
   return sum(model.W[2,t-1], model.W[7,t-1]) <= model.W[11,t] <= C[11]*model.X[12]
model.other_pipe_rule6 = Constraint(model.T, rule=other_pipe_rule6)


ERROR:pyomo.core:Rule failed when generating expression for Constraint inflow_pipe with index 1:
AttributeError: 'ConcreteModel' object has no attribute 'f'
ERROR:pyomo.core:Constructing component 'inflow_pipe' from data=None failed:
    AttributeError: 'ConcreteModel' object has no attribute 'f'


AttributeError: 'ConcreteModel' object has no attribute 'f'

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**